# Лабораторная работа №3

Бельтюков Даниил Евгеньевич M33031

Горин Антон M33031

In [132]:
import numpy as np
import scipy.linalg
import scipy.sparse.linalg
from scipy.sparse import csr_matrix, csc_matrix, eye
from IPython.display import display, HTML
from tabulate import tabulate

In [133]:
def pretty(*args, **kwargs):
    display(HTML(tabulate(*args, **kwargs, tablefmt='html')))

In [134]:
def _check_square(A):
    assert len(A.shape) == 2, "only 2d matrices are supported"
    assert A.shape[0] == A.shape[1], "only nxn matrices are supported"


def lu_doolittle(A):
    _check_square(A)

    n = A.shape[0]

    if isinstance(A, csr_matrix):
        L = eye(n, dtype=np.double, format="csr")
        U = csr_matrix((n, n), dtype=np.double)
    else:
        L = np.eye(n, dtype=np.double)
        U = np.zeros((n, n), dtype=np.double)

    with np.errstate(divide="ignore", invalid="ignore"):
        for k in range(n):
            U[k, k:] = A[k, k:] - L[k, :k] @ U[:k, k:]
            L[(k+1):, k] = (A[(k+1):, k] - L[(k+1):, :] @ U[:, k]) / U[k, k]

        if isinstance(A, np.ndarray):
            L[~np.isfinite(L)] = 0

    return L, U

In [135]:
def noisy_matrix(n):
    np.random.seed(59005)

    matrix = -np.random.choice(5, size=(n, n)).astype(np.double)
    for i in range(n):
        matrix[i, i] = -(np.sum(matrix[i]) - matrix[i, i]) + 10 ** -n

    return matrix


def hilbert_generator(n): return np.fromfunction(lambda i, j: 1 / (i + j + 1),
                                                 (n, n), dtype=np.float)  # since we are starting from i=0 and j=0

In [136]:
def test_lu(A):
    L, U = lu_doolittle(A)
    A_1 = L @ U

    if isinstance(A, csr_matrix):
        A_1 = A_1.A
        n = A.shape[0]
        lu = scipy.sparse.linalg.splu(A)
        Pr = csc_matrix((np.ones(n), (lu.perm_r, np.arange(n))))
        Pc = csc_matrix((np.ones(n), (np.arange(n), lu.perm_c)))
        error_lib = np.linalg.norm(A - (Pr.T * (lu.L * lu.U) * Pc.T).A)
    else:
        P, L, U = scipy.linalg.lu(A)
        error_lib = np.linalg.norm(A - P @ L @ U)

    error_my = np.linalg.norm(A - A_1)
        
    return error_my, error_lib


LU_TESTS = [
    ("LU Decomposition", test_lu, (
        (np.array([[1, 0, 3], [0, 3, 1], [0, 0, 6]]),),
        (np.array([[1, 4, 5], [6, 8, 22], [32, 5, 5]]),),
        (noisy_matrix(5),),
        (noisy_matrix(7),),
        (noisy_matrix(9),),
        (noisy_matrix(11),),
        (noisy_matrix(13),),
        (hilbert_generator(5),),
        (hilbert_generator(7),),
        (hilbert_generator(9),),
        (hilbert_generator(11),),
        (hilbert_generator(13),),
        (csr_matrix(noisy_matrix(11)),)
    ))
]


for name, func, args in LU_TESTS:
    results = []
    print(f"TEST {name}")
    for A in args:
        results.append(func(*A))
    pretty(results, headers=["error_my", "error_lib"])

TEST LU Decomposition


error_my,error_lib
0,0
0,4.44089e-16
1.22933e-15,2.39407e-15
1.97308e-15,2.86255e-15
2.13282e-15,6.78133e-15
6.88354e-15,8.15886e-15
5.14047e-15,1.23981e-14
4.16334e-17,0
4.80741e-17,5.19259e-17
5.76388e-17,7.07631e-17


In [137]:
def forward_substitution(L, b):
    _check_square(L)

    n = L.shape[0]

    if isinstance(L, csr_matrix):
        y = csc_matrix((n, 1), dtype=np.double)
        
    else:
        assert L.shape[0] == b.shape[0]
        y = np.zeros_like(b, dtype=np.double)

    y[0] = b[0] / L[0, 0]

    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]

    return y


def back_substitution(U, y):
    _check_square(U)

    n = U.shape[0]

    if isinstance(U, csr_matrix):
        x = csc_matrix((n, 1), dtype=np.double)
        
    else:
        assert U.shape[0] == y.shape[0]
        x = np.zeros_like(y, dtype=np.double)

    x[-1] = y[-1] / U[-1, -1]

    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i:], x[i:])) / U[i, i]

    return x


def lu_solve(A, b):
    if isinstance(A, np.ndarray):
        assert A.shape[0] == b.shape[0]  # (5, 5) and (5, )

    L, U = lu_doolittle(A)

    y = forward_substitution(L, b)

    return back_substitution(U, y)

In [138]:
def test_solve(A, b):
    x = lu_solve(A, b)
    
    if isinstance(A, csr_matrix):
        error_my = np.linalg.norm((A @ x - b).A)
        error_lib = np.linalg.norm(A @ scipy.sparse.linalg.spsolve(A, b) - b.A.ravel())
    else:
        error_my = np.linalg.norm(A @ x - b)
        error_lib = np.linalg.norm(A @ scipy.linalg.solve(A, b) - b)

    return error_my, error_lib


SOLVE_TESTS = [
    ("LU Solve", test_solve, (
        (
            (np.array([[1, 4, 5], [6, 8, 22], [32, 5, 5]]),
             np.array([1, 2, 3.]))
        ),
        (
            (noisy_matrix(5), np.dot(noisy_matrix(5), np.arange(1, 6)))
        ),
        (
            (noisy_matrix(7), np.dot(noisy_matrix(7), np.arange(1, 8)))
        ),
        (
            (noisy_matrix(9), np.dot(noisy_matrix(9), np.arange(1, 10)))
        ),
        (
            (noisy_matrix(11), np.dot(noisy_matrix(11), np.arange(1, 12)))
        ),
        (
            (noisy_matrix(13), np.dot(noisy_matrix(13), np.arange(1, 14)))
        ),
        (
            (hilbert_generator(5), np.dot(hilbert_generator(5), np.arange(1, 6)))
        ),
        (
            (hilbert_generator(7), np.dot(hilbert_generator(7), np.arange(1, 8)))
        ),
        (
            (hilbert_generator(9), np.dot(hilbert_generator(9), np.arange(1, 10)))
        ),
        (
            (hilbert_generator(11), np.dot(hilbert_generator(11), np.arange(1, 12)))
        ),
        (
            (hilbert_generator(13), np.dot(hilbert_generator(13), np.arange(1, 14)))
        ),
        (
            (csr_matrix(noisy_matrix(5)), csr_matrix(np.dot(noisy_matrix(5), np.arange(1, 6))).T)
        ),
    ))
]


for name, func, args in SOLVE_TESTS:
    results = []
    print(f"TEST {name}")
    for A in args:
        results.append(func(*A))
    pretty(results, headers=["error_my", "error_lib"])

TEST LU Solve


<ipython-input-138-9036bde03575>:9: LinAlgWarning: Ill-conditioned matrix (rcond=1.1721e-18): result may not be accurate.
  error_lib = np.linalg.norm(A @ scipy.linalg.solve(A, b) - b)


error_my,error_lib
0,0
1.37596e-14,1.77636e-15
2.88624e-14,1.28095e-14
4.08176e-14,6.59887e-14
7.41828e-14,9.76835e-14
1.21625e-13,9.31868e-14
0,0
9.93014e-16,9.93014e-16
9.93014e-16,8.88178e-16
1.83103e-15,1.33227e-15


In [139]:
def lu_inverse(A):
    _check_square(A)

    n = A.shape[0]

    if isinstance(A, csr_matrix):
        b = eye(n, dtype=np.double, format="csr")
        A_inv = csr_matrix((n, n), dtype="double")
    else:
        b = np.eye(n, dtype=np.double)
        A_inv = np.zeros_like(A, dtype=np.double)

    L, U = lu_doolittle(A)

    for i in range(n):
        y = forward_substitution(L, b[:, i])
        A_inv[:, i] = back_substitution(U, y)

    return A_inv

In [140]:
def test_inv(A):
    inv = lu_inverse(A)
    
    if isinstance(A, csr_matrix):
        inv_lib = scipy.sparse.linalg.inv(A)
        I = eye(A.shape[0], dtype=np.double, format="csr")
        error_my = np.linalg.norm((A @ inv - I).A)
        error_lib = np.linalg.norm((A @ inv_lib - I).A)
    else:
        # High mistake on small numbers
        # inv_lib = scipy.linalg.inv(A)
        inv_lib = np.linalg.inv(A)
        I = np.eye(A.shape[0])
        error_my = np.linalg.norm(A @ inv - I)
        error_lib = np.linalg.norm(A @ inv_lib - I)
    
    return error_my, error_lib

INV_TESTS = [
    ("LU Inverse", test_inv, (
        (np.array([[1, 0, 3], [0, 3, 1], [0, 0, 6]]),),
        (np.array([[1, 4, 5], [6, 8, 22], [32, 5, 5]]),),
        (noisy_matrix(5),),
        (noisy_matrix(7),),
        (noisy_matrix(9),),
        (noisy_matrix(11),),
        (noisy_matrix(13),),
        (hilbert_generator(5),),
        (hilbert_generator(7),),
        (hilbert_generator(9),),
        (hilbert_generator(11),),
        (hilbert_generator(13),),
        (csr_matrix(noisy_matrix(11)),)
    ))
]


for name, func, args in INV_TESTS:
    results = []
    print(f"TEST {name}")
    for A in args:
        results.append(func(*A))
    pretty(results, headers=["error_my", "error_lib"])

TEST LU Inverse


error_my,error_lib
2.77556e-17,2.77556e-17
2.21384e-15,2.93163e-16
6.66998e-11,8.15414e-11
1.53919e-08,2.11803e-08
2.01844e-06,2.61754e-06
0.000295323,0.000366117
0.0322998,0.0416395
5.45164e-12,3.17921e-12
8.29727e-09,4.60757e-09
6.15661e-06,4.4322e-06


## Вывод

LU-разложение позволяет эффективно решать системы уравнений $Ax = b$ для фиксированной $A$. Быстрее, чем метод Гаусса, а то есть за $O(2n^2)$ вместо $O(\frac{1}{3} n^3)$ на каждую систему.

Нахождение обратной матрицы с помощью LU-разложения находится за $O(n^3)$.

Стоит отметить, что L2 норма ошибки растет пропорционально увеличению размерности матрицы. Такое поведение наблюдается аналогично в реализации методов из библиотек `numpy` и `scipy`.

Все реализованные методы поддерживают работу с разреженными матрицами в CSR формате.